# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [110]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy.stats import percentileofscore as score

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [111]:
stocks = pd.read_csv('sp_500_stocks.csv')
from iextoken import IEX_CLOUD_API_TOKEN


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [112]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2732707411280,
 'week52high': 197.7,
 'week52low': 123.48,
 'week52highSplitAdjustOnly': 198.23,
 'week52lowSplitAdjustOnly': 124.17,
 'week52change': 0.1718846901401181,
 'sharesOutstanding': 15634232000,
 'float': 0,
 'avg10Volume': 67775776,
 'avg30Volume': 66866726,
 'day200MovingAvg': 175.91,
 'day50MovingAvg': 178.86,
 'employees': 164000,
 'ttmEPS': 5.95,
 'ttmDividendRate': 0.9368421399305272,
 'dividendYield': 0.005359815435268192,
 'nextDividendDate': '',
 'exDividendDate': '2023-08-11',
 'nextEarningsDate': '2023-11-02',
 'peRatio': 28.838195560151963,
 'beta': 1.3271866325627353,
 'maxChangePercent': 67.13893653516296,
 'year5ChangePercent': 2.362691230338289,
 'year2ChangePercent': 0.20734422261717175,
 'year1ChangePercent': 0.1718846901401181,
 'ytdChangePercent': 0.3528260822453864,
 'month6ChangePercent': 0.10428998241753518,
 'month3ChangePercent': -0.06116172307090728,
 'month1ChangePercent': -0.03494920494699649,
 'day30Chan

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [113]:
data['year1ChangePercent']

0.1718846901401181

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [114]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [115]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_url = 'https://cloud.iexapis.com/stable/stock/market/batch?symbols={}&types=stats,price&token={}'.format(symbol_string, IEX_CLOUD_API_TOKEN)
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        try:
            final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A'
                ],
                index=my_columns
            ),
            ignore_index=True
            )
        except:
            print(f'No data on {symbol}')
       

C:\Users\user\AppData\Local\Temp\ipykernel_5256\637844692.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_5256\637844692.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_5256\637844692.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_5256\637844692.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipy

No data on AET
No data on GAS
No data on ARG
No data on ANR
No data on ABC
No data on APOL
No data on BHI
No data on BCR
No data on BMC
No data on BRCM
No data on CA
No data on CVC
No data on CAM
No data on CFN
No data on CBG


C:\Users\user\AppData\Local\Temp\ipykernel_5256\637844692.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_5256\637844692.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_5256\637844692.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_5256\637844692.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipy

No data on COH
No data on CCE
No data on CSC
No data on CBE
No data on CVH
No data on COV
No data on DISCA
No data on DPS
No data on ESRX
No data on FDO
No data on FII


C:\Users\user\AppData\Local\Temp\ipykernel_5256\637844692.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_5256\637844692.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_5256\637844692.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_5256\637844692.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipy

No data on HAR
No data on HCN
No data on HNZ
No data on HSP
No data on HCBK
No data on TEG
No data on JEC
No data on JDSU
No data on JOY
No data on KFT
No data on LUK
No data on LXK
No data on LTD
No data on LLTC
No data on LO
No data on MHP
No data on MJN
No data on MWV
No data on PCS
No data on MOLX
No data on POM
No data on PCL
No data on PCP
No data on PCLN
No data on RAI
No data on COL
No data on SWY
No data on SNDK
No data on SCG


C:\Users\user\AppData\Local\Temp\ipykernel_5256\637844692.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_5256\637844692.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_5256\637844692.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_5256\637844692.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipy

No data on SNI
No data on SIAL
No data on STJ
No data on SPLS
No data on HOT
No data on SYMC
No data on TE
No data on TSO
No data on TWX
No data on TWC
No data on TIE
No data on TYC
No data on WAG
No data on WPO
No data on WPI
No data on WLP
No data on WFM
No data on WYN
No data on YHOO
No data on ZMH


C:\Users\user\AppData\Local\Temp\ipykernel_5256\637844692.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_5256\637844692.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_5256\637844692.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_5256\637844692.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipy

## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [116]:
final_dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)
#inplace = true modifies the final_dataframe object

final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)

## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [117]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio:')

    try:
        float(portfolio_size)
    except:
        print('That is not a number! \nPlease try again:')
        portfolio_size = input('Enter the size of your portfolio:')

portfolio_input()
print(portfolio_size)

1000000


In [118]:
#position size
position_size = float(portfolio_size)/len(final_dataframe.index)
print(position_size)

for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])

final_dataframe

20000.0


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,328,QEP,4.08,6.266251,4901
1,126,RRD,10.84,2.419558,1845
2,289,NVDA,416.1,2.326578,48
3,2,ANF,50.63,2.188287,395
4,167,FTI,20.85,1.503873,959
5,121,DO,14.55,1.298578,1374
6,178,GE,111.25,1.198395,179
7,208,IGT,30.93,1.067804,646
8,116,DELL,70.05,1.038915,285
9,215,JBL,107.6,0.920767,185


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [119]:
hqm_columns = ['Ticker', 'Price', 'Number of Shares to Buy', 'One Year Price Return', 'One Year Return Percentile', 'Six Month Price Return', 'Six Month Return Percentile', 'Three Month Price Return', 'Three Month Return Percentile', 'One Month Price Return', 'One Month Return Percentile', 'HQM Score']
hqm_dataframe = pd.DataFrame(columns=hqm_columns)


for symbol_string in symbol_strings:
    batch_api_url = 'https://cloud.iexapis.com/stable/stock/market/batch?symbols={}&types=stats,price&token={}'.format(symbol_string, IEX_CLOUD_API_TOKEN)
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        try:
            hqm_dataframe = hqm_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    'N/A',
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month6ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month3ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month1ChangePercent'],
                    'N/A',
                    'N/A'
                    
                ],
                index=hqm_columns
            ),
            ignore_index=True
            )
        except:
            print(f'No data on {symbol}')

hqm_dataframe

C:\Users\user\AppData\Local\Temp\ipykernel_5256\3488662888.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_5256\3488662888.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_5256\3488662888.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_5256\3488662888.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_5

No data on AET
No data on GAS
No data on ARG
No data on ANR
No data on ABC
No data on APOL
No data on BHI
No data on BCR
No data on BMC
No data on BRCM
No data on CA
No data on CVC
No data on CAM
No data on CFN
No data on CBG


C:\Users\user\AppData\Local\Temp\ipykernel_5256\3488662888.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_5256\3488662888.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_5256\3488662888.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_5256\3488662888.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_5

No data on COH
No data on CCE
No data on CSC
No data on CBE
No data on CVH
No data on COV
No data on DISCA
No data on DPS
No data on ESRX
No data on FDO
No data on FII


C:\Users\user\AppData\Local\Temp\ipykernel_5256\3488662888.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_5256\3488662888.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_5256\3488662888.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_5256\3488662888.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_5

No data on HAR
No data on HCN
No data on HNZ
No data on HSP
No data on HCBK
No data on TEG
No data on JEC
No data on JDSU
No data on JOY
No data on KFT
No data on LUK
No data on LXK
No data on LTD
No data on LLTC
No data on LO
No data on MHP
No data on MJN
No data on MWV
No data on PCS
No data on MOLX
No data on POM
No data on PCL
No data on PCP
No data on PCLN
No data on RAI
No data on COL
No data on SWY
No data on SNDK
No data on SCG


C:\Users\user\AppData\Local\Temp\ipykernel_5256\3488662888.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_5256\3488662888.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_5256\3488662888.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_5256\3488662888.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_5

No data on SNI
No data on SIAL
No data on STJ
No data on SPLS
No data on HOT
No data on SYMC
No data on TE
No data on TSO
No data on TWX
No data on TWC
No data on TIE
No data on TYC
No data on WAG
No data on WPO
No data on WPI
No data on WLP
No data on WFM
No data on WYN
No data on YHOO
No data on ZMH


C:\Users\user\AppData\Local\Temp\ipykernel_5256\3488662888.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_5256\3488662888.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_5256\3488662888.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_5256\3488662888.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
C:\Users\user\AppData\Local\Temp\ipykernel_5

,Ticker,Price,Number of Shares to Buy,One Year Price Return,One Year Return Percentile,Six Month Price Return,Six Month Return Percentile,Three Month Price Return,Three Month Return Percentile,One Month Price Return,One Month Return Percentile,HQM Score
0,MMM,97.04,N/A,-0.092853,N/A,-0.010016,N/A,-0.022128,N/A,-0.026192,N/A,N/A
1,ABT,98.15,N/A,-0.006435,N/A,0.021368,N/A,-0.087268,N/A,-0.066394,N/A,N/A
2,ANF,50.63,N/A,2.188287,N/A,1.056458,N/A,0.370972,N/A,-0.004522,N/A,N/A
3,ACN,316.09,N/A,0.235509,N/A,0.172358,N/A,0.063021,N/A,0.002378,N/A,N/A
4,ADBE,512.9,N/A,0.802432,N/A,0.389973,N/A,0.058137,N/A,-0.033559,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
419,XLNX,194.92,N/A,0,N/A,0,N/A,0,N/A,0,N/A,N/A
420,XL,1.26,N/A,0,N/A,0,N/A,0,N/A,0,N/A,N/A
421,XYL,90.9,N/A,0.024115,N/A,-0.058937,N/A,-0.178653,N/A,-0.093226,N/A,N/A
422,YUM,125,N/A,0.150776,N/A,-0.011814,N/A,-0.073852,N/A,-0.029477,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [120]:
time_periods = [
    'One Year',
    'Six Month',
    'Three Month',
    'One Month'
]



hqm_dataframe = hqm_dataframe.dropna(subset=['One Year Price Return', 'Six Month Price Return', 'Three Month Price Return', 'One Month Price Return'])


#updated hqm_dataframe without N/A values


for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100

hqm_dataframe

        

C:\Users\user\AppData\Local\Temp\ipykernel_5256\4047322649.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100
C:\Users\user\AppData\Local\Temp\ipykernel_5256\4047322649.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100
C:\Users\user\AppData\Local\Temp\ipykernel_5256\4047322649.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats 

,Ticker,Price,Number of Shares to Buy,One Year Price Return,One Year Return Percentile,Six Month Price Return,Six Month Return Percentile,Three Month Price Return,Three Month Return Percentile,One Month Price Return,One Month Return Percentile,HQM Score
0,MMM,97.04,N/A,-0.092853,0.199495,-0.010016,0.305556,-0.022128,0.416667,-0.026192,0.444444,N/A
1,ABT,98.15,N/A,-0.006435,0.320707,0.021368,0.409091,-0.087268,0.184343,-0.066394,0.199495,N/A
2,ANF,50.63,N/A,2.188287,0.992424,1.056458,0.994949,0.370972,0.979798,-0.004522,0.603535,N/A
3,ACN,316.09,N/A,0.235509,0.681818,0.172358,0.737374,0.063021,0.742424,0.002378,0.691919,N/A
4,ADBE,512.9,N/A,0.802432,0.972222,0.389973,0.964646,0.058137,0.722222,-0.033559,0.383838,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
419,XLNX,194.92,N/A,0,0.342172,0,0.344697,0,0.516414,0,0.660354,N/A
420,XL,1.26,N/A,0,0.342172,0,0.344697,0,0.516414,0,0.660354,N/A
421,XYL,90.9,N/A,0.024115,0.396465,-0.058937,0.179293,-0.178653,0.063131,-0.093226,0.10101,N/A
422,YUM,125,N/A,0.150776,0.573232,-0.011814,0.295455,-0.073852,0.224747,-0.029477,0.421717,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [121]:
from statistics import mean 
for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
    
hqm_dataframe
        

C:\Users\user\AppData\Local\Temp\ipykernel_5256\3987723607.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)


,Ticker,Price,Number of Shares to Buy,One Year Price Return,One Year Return Percentile,Six Month Price Return,Six Month Return Percentile,Three Month Price Return,Three Month Return Percentile,One Month Price Return,One Month Return Percentile,HQM Score
0,MMM,97.04,N/A,-0.092853,0.199495,-0.010016,0.305556,-0.022128,0.416667,-0.026192,0.444444,0.34154
1,ABT,98.15,N/A,-0.006435,0.320707,0.021368,0.409091,-0.087268,0.184343,-0.066394,0.199495,0.278409
2,ANF,50.63,N/A,2.188287,0.992424,1.056458,0.994949,0.370972,0.979798,-0.004522,0.603535,0.892677
3,ACN,316.09,N/A,0.235509,0.681818,0.172358,0.737374,0.063021,0.742424,0.002378,0.691919,0.713384
4,ADBE,512.9,N/A,0.802432,0.972222,0.389973,0.964646,0.058137,0.722222,-0.033559,0.383838,0.760732
...,...,...,...,...,...,...,...,...,...,...,...,...
419,XLNX,194.92,N/A,0,0.342172,0,0.344697,0,0.516414,0,0.660354,0.465909
420,XL,1.26,N/A,0,0.342172,0,0.344697,0,0.516414,0,0.660354,0.465909
421,XYL,90.9,N/A,0.024115,0.396465,-0.058937,0.179293,-0.178653,0.063131,-0.093226,0.10101,0.184975
422,YUM,125,N/A,0.150776,0.573232,-0.011814,0.295455,-0.073852,0.224747,-0.029477,0.421717,0.378788


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [122]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop = True, inplace=True)
hqm_dataframe   

C:\Users\user\AppData\Local\Temp\ipykernel_5256\4214540940.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)


,Ticker,Price,Number of Shares to Buy,One Year Price Return,One Year Return Percentile,Six Month Price Return,Six Month Return Percentile,Three Month Price Return,Three Month Return Percentile,One Month Price Return,One Month Return Percentile,HQM Score
0,QEP,4.08,N/A,6.266251,1.0,2.578947,1.0,0.73617,0.997475,0.182609,0.992424,0.997475
1,DELL,70.05,N/A,1.038915,0.979798,0.881224,0.992424,0.386618,0.987374,0.232624,0.997475,0.989268
2,FTI,20.85,N/A,1.503873,0.989899,0.664591,0.984848,0.415517,0.992424,0.133152,0.989899,0.989268
3,HAL,40.63,N/A,0.680989,0.949495,0.393763,0.967172,0.320618,0.974747,0.053562,0.929293,0.955177
4,PSX,118.58,N/A,0.622699,0.934343,0.293663,0.906566,0.317296,0.972222,0.056298,0.936869,0.9375
5,MPC,153.46,N/A,0.718756,0.962121,0.231007,0.835859,0.381573,0.984848,0.071274,0.957071,0.934975
6,FLR,36.87,N/A,0.489697,0.893939,0.254082,0.863636,0.299612,0.962121,0.113225,0.977273,0.924242
7,R,103.78,N/A,0.596333,0.926768,0.244567,0.85101,0.27466,0.95202,0.071222,0.954545,0.921086
8,SLB,59.04,N/A,0.717018,0.957071,0.310663,0.929293,0.272515,0.949495,0.024123,0.815657,0.912879
9,X,32,N/A,0.736695,0.964646,0.285709,0.893939,0.374458,0.982323,0.021711,0.810606,0.912879


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [123]:
portfolio_input()

In [124]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])
    
hqm_dataframe

C:\Users\user\AppData\Local\Temp\ipykernel_5256\656535724.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])


,Ticker,Price,Number of Shares to Buy,One Year Price Return,One Year Return Percentile,Six Month Price Return,Six Month Return Percentile,Three Month Price Return,Three Month Return Percentile,One Month Price Return,One Month Return Percentile,HQM Score
0,QEP,4.08,4901,6.266251,1.0,2.578947,1.0,0.73617,0.997475,0.182609,0.992424,0.997475
1,DELL,70.05,285,1.038915,0.979798,0.881224,0.992424,0.386618,0.987374,0.232624,0.997475,0.989268
2,FTI,20.85,959,1.503873,0.989899,0.664591,0.984848,0.415517,0.992424,0.133152,0.989899,0.989268
3,HAL,40.63,492,0.680989,0.949495,0.393763,0.967172,0.320618,0.974747,0.053562,0.929293,0.955177
4,PSX,118.58,168,0.622699,0.934343,0.293663,0.906566,0.317296,0.972222,0.056298,0.936869,0.9375
5,MPC,153.46,130,0.718756,0.962121,0.231007,0.835859,0.381573,0.984848,0.071274,0.957071,0.934975
6,FLR,36.87,542,0.489697,0.893939,0.254082,0.863636,0.299612,0.962121,0.113225,0.977273,0.924242
7,R,103.78,192,0.596333,0.926768,0.244567,0.85101,0.27466,0.95202,0.071222,0.954545,0.921086
8,SLB,59.04,338,0.717018,0.957071,0.310663,0.929293,0.272515,0.949495,0.024123,0.815657,0.912879
9,X,32,625,0.736695,0.964646,0.285709,0.893939,0.374458,0.982323,0.021711,0.810606,0.912879


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [125]:
writer = pd.ExcelWriter('Momentum Strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index=False)


## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [126]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [127]:
column_formats = {
        'A': ['Ticker', string_template],
        'B': ['Price', dollar_template],
        'C': ['Number of Shares to Buy', integer_template],
        'D': ['One Year Price Return', percent_template],
        'E': ['One Year Return Percentile', percent_template],
        'F': ['Six Month Price Return', percent_template],
        'G': ['Six Month Return Percentile', percent_template],
        'H': ['Three Month Price Return', percent_template],
        'I': ['Three Month Return Percentile', percent_template],
        'J': ['One Month Price Return', percent_template],
        'K': ['One Month Return Percentile', percent_template],
        'L': ['HQM Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [128]:
writer.save()

C:\Users\user\AppData\Local\Temp\ipykernel_5256\934276808.py:1: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
